# Usage example

This notebook aims at providing some example code on how to use the python package PyPowerGadget.
This package allows you to monitor the energy consumption and CO2 footprint of any sample of code or function

[TOC]

In all this notebook we'll try to measure how much CO2 does the training of a toy algorithm on the mnist database emit
```python
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
clf = SGDClassifier(loss='log', alpha=1e-5)
cv = StratifiedKFold(10, random_state=0, shuffle=True)
cv_results = cross_validate(clf, X, y, cv=cv)
print(cv_results['test_score'].mean(), cv_results['test_score'].std())
```

In [1]:
from PyPowerGadget import PowerMeter

In [2]:
help(PowerMeter)

Help on class PowerMeter in module PyPowerGadget.PowerMeter:

class PowerMeter(builtins.object)
 |  PowerMeter(project_name='', cpu_power_log_path='', get_country=True, user_name='')
 |  
 |  PowerMeter is a general tool to monitor and log the power consumption of any given function.
 |  
 |  Parameters
 |  ----------
 |  project_name (optional) : str
 |      Name of the project you are working on (default is folder_name)
 |  cpu_power_log_path (optional) : str
 |      The path to the tool "PowerLog"
 |  get_country (optional) : bool
 |      Whether use user country location or not
 |  user_name (optional) : str
 |      The name of the user using the tool (for logging purpose)
 |  
 |  Methods defined here:
 |  
 |  __call__(self, package, algorithm, data_type='', data_shape='', algorithm_params='', comments='')
 |      Call self as a function.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, type, value, traceback)
 |  
 |  __init__(self, project_name='', cpu_power_log_path='', get_co

In [3]:
# imports
from PyPowerGadget import PowerMeter #Our package

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import SGDClassifier

In [4]:
# Creates a power meter object that contains information relative to the current project
# You need to do this step no matter how you use the package
power_meter = PowerMeter(project_name="Usage examples")

## Function decorator

In [5]:
# Add a decorator to the main function to measure power usage of this function each time it is called
@power_meter.measure_power(
  package="sklearn",
  algorithm="SGDClassifier",
  data_type="tabular/images",
  data_shape="(1797, 64)",
  algorithm_params="loss='log', alpha=1e-5",
  comments="10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
)
def cross_val_mnist(alpha, random_state=0):
    mnist = datasets.load_digits()
    X = mnist.data
    y = mnist.target
    clf = SGDClassifier(loss='log', alpha=alpha, random_state=random_state)
    cv = StratifiedKFold(10, random_state=random_state, shuffle=True)
    cv_results = cross_validate(clf, X, y, cv=cv)
    print(cv_results['test_score'].mean(), cv_results['test_score'].std())
    return cv_results

In [6]:
train_results = cross_val_mnist(1e-5)

starting CPU power monitoring ...
0.9565983860955928 0.013556136197153262
stoping CPU power monitoring ...
This process emitted 0.001g of CO2 (using the energy mix of b'France')


## Magic function
Only usable in notebooks

In [7]:
# import the magic function module
%load_ext PyPowerGadget.MagicPowerMeter

In [8]:
%%measure_power?

In [9]:
%%measure_power power_meter "sklearn" "SGDClassifier" --data_type "tabular/images" --data_shape "(1797,64)" --algorithm_params "loss='log', alpha=1e-5" --comments "10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
# This command must always be on the first line of the cell and must be written on one line
mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
clf = SGDClassifier(loss='log', alpha=1e-5, random_state=0)
cv = StratifiedKFold(10, random_state=0, shuffle=True)
cv_results = cross_validate(clf, X, y, cv=cv)
print(cv_results['test_score'].mean(), cv_results['test_score'].std())

starting CPU power monitoring ...
0.9565983860955928 0.013556136197153262
stoping CPU power monitoring ...
This process emitted 0.000g of CO2 (using the energy mix of b'France')


## Inline commands
There are 2 ways of using this package using inline commands, we recommmand using the *with statement* over the *start and stop methods* as the with statement stops the process automatically even if you get an error in the process

### Using the with statement

In [10]:
with power_meter(
  package="sklearn",
  algorithm="SGDClassifier",
  data_type="tabular/images",
  data_shape="(1797, 64)",
  algorithm_params="loss='log', alpha=1e-5",
  comments="10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
):
    mnist = datasets.load_digits()
    X = mnist.data
    y = mnist.target
    clf = SGDClassifier(loss='log', alpha=1e-5, random_state=0)
    cv = StratifiedKFold(10, random_state=0, shuffle=True)
    cv_results = cross_validate(clf, X, y, cv=cv)
    print(cv_results['test_score'].mean(), cv_results['test_score'].std())

starting CPU power monitoring ...
0.9565983860955928 0.013556136197153262
stoping CPU power monitoring ...
This process emitted 0.001g of CO2 (using the energy mix of b'France')


### Using start_measure and stop_measure methods

In [11]:
power_meter.start_measure(
  package="sklearn",
  algorithm="SGDClassifier",
  data_type="tabular/images",
  data_shape="(1797, 64)",
  algorithm_params="loss='log', alpha=1e-5",
  comments="10 fold cross validated training of logistic regression classifier trained on the MNIST dataset"
)

mnist = datasets.load_digits()
X = mnist.data
y = mnist.target
clf = SGDClassifier(loss='log', alpha=1e-5, random_state=0)
cv = StratifiedKFold(10, random_state=0, shuffle=True)
cv_results = cross_validate(clf, X, y, cv=cv)
print(cv_results['test_score'].mean(), cv_results['test_score'].std())

power_meter.stop_measure()

starting CPU power monitoring ...
0.9565983860955928 0.013556136197153262
stoping CPU power monitoring ...
This process emitted 0.001g of CO2 (using the energy mix of b'France')
